In [ ]:
%pip install sagemaker -qU

In [ ]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

# Demand Forecasting with XGBoost on Amazon SageMaker

This notebook demonstrates how to:
1. Prepare a synthetic demand forecasting dataset
2. Train an XGBoost model on Amazon SageMaker
3. Deploy the model to a SageMaker endpoint for real-time inference

The workflow includes:
- Generating synthetic time series data with trend, seasonality, and noise
- Feature engineering for time series forecasting
- Training an XGBoost model on SageMaker
- Deploying the model to a SageMaker endpoint
- Testing the endpoint with sample data

## Setup and Imports

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import os

## Initialize SageMaker Session

In [ ]:
# Initialize SageMaker session
session = sagemaker.Session()
bucket = session.default_bucket()
region = boto3.Session().region_name

try:
    role = sagemaker.get_execution_role()
except:
    role = input("> Enter the role ARN for your SageMaker execution role: ")
    role = role.strip()

print(f"SageMaker session initialized in region: {region}")
print(f"Using S3 bucket: {bucket}")
print(f"Using IAM role: {role}")

## Generate Synthetic Demand Data

We'll create synthetic time series data with trend, seasonality, and noise components to simulate demand patterns.

In [ ]:
def generate_demand_data(n_periods=365*2, seasonality=True, trend=True, noise_level=0.2):
    """Generate synthetic demand data with trend, seasonality, and noise."""
    time_idx = np.arange(n_periods)
    
    # Base demand
    base_demand = 100
    
    # Add trend component
    trend_component = 0
    if trend:
        trend_component = time_idx * 0.1
    
    # Add seasonality component
    seasonality_component = 0
    if seasonality:
        # Weekly seasonality
        weekly = 10 * np.sin(2 * np.pi * time_idx / 7)
        # Monthly seasonality
        monthly = 20 * np.sin(2 * np.pi * time_idx / 30)
        # Yearly seasonality
        yearly = 50 * np.sin(2 * np.pi * time_idx / 365)
        
        seasonality_component = weekly + monthly + yearly
    
    # Add noise
    noise = np.random.normal(0, noise_level * base_demand, n_periods)
    
    # Combine components
    demand = base_demand + trend_component + seasonality_component + noise
    
    # Ensure no negative values
    demand = np.maximum(demand, 0)
    
    # Create DataFrame
    dates = pd.date_range(start='2021-01-01', periods=n_periods)
    df = pd.DataFrame({
        'date': dates,
        'demand': demand
    })
    
    # Add date features
    df['dayofweek'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['year'] = df['date'].dt.year
    df['quarter'] = df['date'].dt.quarter
    
    return df

# Generate data
print("Generating synthetic demand forecasting data...")
df = generate_demand_data()
print(f"Dataset shape: {df.shape}")
df.head()

## Visualize the Data

Let's visualize the synthetic demand data to understand its patterns.

In [ ]:
def visualize_data(df):
    """Visualize the demand data."""
    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['demand'])
    plt.title('Synthetic Demand Data')
    plt.xlabel('Date')
    plt.ylabel('Demand')
    plt.grid(True)
    plt.tight_layout()
    return plt

# Visualize data
visualize_data(df)

## Feature Engineering

Create lag and rolling window features for time series forecasting.

In [ ]:
def prepare_features(df):
    """Create lag and rolling features for time series forecasting."""
    # Create lag features (previous demand values)
    for lag in [1, 7, 14, 30]:
        df[f'lag_{lag}'] = df['demand'].shift(lag)

    # Create rolling window features
    for window in [7, 14, 30]:
        df[f'rolling_mean_{window}'] = df['demand'].rolling(window=window).mean()
        df[f'rolling_std_{window}'] = df['demand'].rolling(window=window).std()

    # Drop rows with NaN values (due to lag and rolling features)
    df = df.dropna()
    
    return df

# Prepare features
df = prepare_features(df)
print(f"Dataset shape after feature engineering: {df.shape}")
df.head()

## Split Data into Train, Validation, and Test Sets

In [ ]:
def split_data(df):
    """Split data into features and target, then into train/val/test sets."""
    # Define features and target
    features = [col for col in df.columns if col not in ['date', 'demand']]
    X = df[features]
    y = df['demand']

    # Split data into train, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

# Split data
X_train, y_train, X_val, y_val, X_test, y_test = split_data(df)
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

## Save Data to S3

Save the datasets to CSV files and upload them to S3 for SageMaker training.

In [ ]:
def save_data_to_s3(X_train, y_train, X_val, y_val, X_test, y_test, session, bucket):
    """Save datasets to CSV and upload to S3."""
    # Create local directories for data
    os.makedirs('data', exist_ok=True)

    # Save datasets to CSV
    train_data = pd.concat([X_train, y_train], axis=1)
    val_data = pd.concat([X_val, y_val], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)

    train_data.to_csv('data/train.csv', index=False)
    val_data.to_csv('data/validation.csv', index=False)
    test_data.to_csv('data/test.csv', index=False)

    # Upload to S3
    train_s3_path = session.upload_data('data/train.csv', bucket=bucket, key_prefix='demand-forecast/data')
    val_s3_path = session.upload_data('data/validation.csv', bucket=bucket, key_prefix='demand-forecast/data')
    test_s3_path = session.upload_data('data/test.csv', bucket=bucket, key_prefix='demand-forecast/data')
    
    return train_s3_path, val_s3_path, test_s3_path

# Save data to S3
train_s3_path, val_s3_path, test_s3_path = save_data_to_s3(
    X_train, y_train, X_val, y_val, X_test, y_test, session, bucket
)
print(f"Training data uploaded to: {train_s3_path}")
print(f"Validation data uploaded to: {val_s3_path}")
print(f"Test data uploaded to: {test_s3_path}")

## Train XGBoost Model on SageMaker

Define and train an XGBoost model using SageMaker's training infrastructure.

In [ ]:
def train_model(train_s3_path, val_s3_path, role, bucket, instance_type='ml.m5.4xlarge'):
    """Train XGBoost model on SageMaker."""
    # Define XGBoost hyperparameters
    hyperparameters = {
        'max_depth': 6,
        'eta': 0.2,
        'gamma': 4,
        'min_child_weight': 6,
        'subsample': 0.8,
        'objective': 'reg:squarederror',
        'num_round': 100,
        'verbosity': 1
    }

    # Create XGBoost estimator
    xgb_estimator = XGBoost(
        entry_point='training_script.py', source_dir="utils",
        framework_version='1.5-1',
        hyperparameters=hyperparameters,
        role=role,
        instance_count=1,
        instance_type=instance_type,
        output_path=f's3://{bucket}/demand-forecast/output',
        keep_alive_period_in_seconds=3600,
    )

    # Train the model
    print("Training XGBoost model...")
    xgb_estimator.fit({'train': train_s3_path, 'validation': val_s3_path})
    
    return xgb_estimator

# Train model (Note: This cell will take some time to execute)
# You can change the instance_type parameter to use a different instance type
instance_type = 'ml.m5.4xlarge'  # You can change this to a smaller instance if needed
xgb_estimator = train_model(train_s3_path, val_s3_path, role, bucket, instance_type)

## Deploy Model to SageMaker Endpoint

Deploy the trained model to a SageMaker endpoint for real-time inference.

In [ ]:
from sagemaker.serializers import JSONSerializer
import boto3

def deploy_model(predictor, instance_type='ml.m5.xlarge'):
    """Deploy model to SageMaker endpoint."""
    # Delete endpoint configuration `ml-models-as-tools` with boto3 
    try:
        sagemaker_client = boto3.client('sagemaker')
        sagemaker_client.delete_endpoint_config(EndpointConfigName='ml-models-as-tools')
    except:
        pass

    # Deploy the endpoint
    print("Deploying model to SageMaker endpoint...")
    predictor = xgb_estimator.deploy(
        endpoint_name="ml-models-as-tools",
        initial_instance_count=1,
        instance_type=instance_type,
        serializer=JSONSerializer(),
        deserializer=JSONDeserializer()
    )
    
    endpoint_name = predictor.endpoint_name
    print(f"Model deployed to endpoint: {endpoint_name}")
    
    return predictor

# Deploy model
# You can change the instance_type parameter to use a different instance type
deploy_instance_type = 'ml.m5.xlarge'  # You can change this to a smaller instance if needed
predictor = deploy_model(xgb_estimator, deploy_instance_type)

## Test the Endpoint

Test the deployed endpoint with sample data from the test set.

In [ ]:
test_sample = X_test.iloc[:5].values.tolist()
actual = y_test.iloc[:5].values

In [ ]:
import boto3, json

def generate_prediction_with_boto3(endpoint_name, test_sample):
    sagemaker_runtime = boto3.client("sagemaker-runtime")
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(test_sample),
        ContentType="application/json",
        Accept="application/json"
    )
    predictions = json.loads(response['Body'].read().decode("utf-8"))
    return np.array(predictions)

generate_prediction_with_boto3("ml-models-as-tools", test_sample)

In [ ]:
def test_endpoint(predictor, test_sample, actual):
    """Test the endpoint with sample data."""

    # Make predictions
    print("Testing the endpoint with sample data...")
    predictions = predictor.predict(test_sample)
    predicted = np.array(predictions)

    # Compare predictions with actual values
    print("Sample predictions:")
    for i in range(len(actual)):
        print(f"Actual: {actual[i]:.2f}, Predicted: {predicted[i]:.2f}")

    # Calculate error metrics
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    print(f"Mean Absolute Error: {mae:.2f}")
    print(f"Root Mean Squared Error: {rmse:.2f}")
    
    return mae, rmse

# Test endpoint
mae, rmse = test_endpoint(predictor, test_sample, actual)

## Visualize Predictions

Let's visualize the predictions against actual values for a larger portion of the test set.

In [ ]:
# Get predictions for more test samples
n_samples = 50  # Number of samples to predict
test_samples = X_test.iloc[:n_samples].values.tolist()
predictions = predictor.predict(test_samples)
actual = y_test.iloc[:n_samples].values

# Plot actual vs predicted
plt.figure(figsize=(12, 6))
plt.plot(range(len(actual)), actual, label='Actual')
plt.plot(range(len(predictions)), predictions, label='Predicted')
plt.title('Actual vs Predicted Demand')
plt.xlabel('Sample Index')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.tight_layout()

## Store info for later use

In [ ]:
ENDPOINT_NAME = predictor.endpoint_name
%store ENDPOINT_NAME test_sample actual

----

## Conclusion

In this notebook, we've demonstrated how to:
1. Generate synthetic demand data with trend, seasonality, and noise
2. Create time series features using lag and rolling window techniques
3. Train an XGBoost model on Amazon SageMaker
4. Deploy the model to a SageMaker endpoint
5. Test the endpoint with real-time predictions

This workflow can be adapted for real-world demand forecasting applications by replacing the synthetic data with actual historical demand data.